In [2]:
import pandas as pd

In [3]:
df_pre = pd.read_csv('Resultados comparativa anterior.csv')
df_pre

,NOMBRE DEL INVESTIGADOR,NOMBRE DEL INVESTIGADOR PATENTE,SIMILARITY,NOMBRE_INVESTIGADOR_NORM,NOMBRE_PATENTE_NORM,ES_MISMA_PERSONA,SCORE,DETALLES
0,"ABSALON CONSTANTINO,ANGEL EDUARDO","ABSALON CONSTANTINO,ANGEL EDUARDO",100,"ABSALON CONSTANTINO,ANGEL EDUARDO","ABSALON CONSTANTINO,ANGEL EDUARDO",False,40.0,Similitud Nombre (100%)
1,"ABURTO ANELL,JORGE ARTURO","ABURTO ANELL,JORGE ARTURO",100,"ABURTO ANELL,JORGE ARTURO","ABURTO ANELL,JORGE ARTURO",False,60.0,Institución
2,"ACEVES DIEZ,ANGEL EMILIO","ACEVES DIEZ,ANGEL EMILIO",100,"ACEVES DIEZ,ANGEL EMILIO","ACEVES DIEZ,ANGEL EMILIO",False,40.0,Sin coincidencias
3,"RIVAS ACEVES,SALVADOR","ACEVES,SALVADOR M",7.777.777.777.777.770,"RIVAS ACEVES,SALVADOR","ACEVES,SALVADOR M",False,37.6,Similitud Nombre (94%)
4,"ACOSTA GONZALEZ,GUILLERMO ENRIQUE","ACOSTA GONZALEZ,GUILLERMO ENRIQUE",100,"ACOSTA GONZALEZ,GUILLERMO ENRIQUE","ACOSTA GONZALEZ,GUILLERMO ENRIQUE",False,40.0,Sin coincidencias
...,...,...,...,...,...,...,...,...
971,"ZAVALA OLIVARES,GERARDO","ZAVALA OLIVARES,GERARDO",100,"ZAVALA OLIVARES,GERARDO","ZAVALA OLIVARES,GERARDO",False,60.0,Institución
972,"ZAVALA YOE,RICARDO","ZAVALA,RICARDO",87.5,"ZAVALA YOE,RICARDO","ZAVALA,RICARDO",True,80.0,"Institución, Fecha"
973,"ZENTENO RAMIREZ,ANTONIO","ZENTENO RAMIREZ,ANTONIO",100,"ZENTENO RAMIREZ,ANTONIO","ZENTENO RAMIREZ,ANTONIO",False,40.0,Sin coincidencias
974,"ZEPEDA PARTIDA,TRINO ARMANDO","ZEPEDA PARTIDA,TRINO ARMANDO",100,"ZEPEDA PARTIDA,TRINO ARMANDO","ZEPEDA PARTIDA,TRINO ARMANDO",False,60.0,Institución


In [4]:
# Convertimos a float la columna SIMILARITY, los números con más de un punto como '7.777.777.777.777.770' agarrar solo los primeros 3 dígitos '7.777.777.777.777.770' -> '7.77'
def formatear_similarity(valor):
    cad = str(valor)
    idx = cad.find('.')
    if idx != -1:
        # Toma los primeros dos dígitos antes del punto
        entero = cad[:idx].replace('.', '')[:2]
        # Toma el primer dígito después del punto
        decimal = cad[idx+1:idx+2]
        fixed = f"{entero}.{decimal}"
        try:
            resultado = float(fixed)
            return min(resultado, 100.0)  # Limitar el valor máximo a 100
        except ValueError:
            return 0.0  # o maneja el error según convenga
    return float(cad)

df_pre['SIMILARITY'] = df_pre['SIMILARITY'].apply(formatear_similarity)

In [7]:
# guardamos
df_pre.to_csv('Resultados comparativa anterior.csv', index=False)

In [ ]:
# eliminamos los datos que tengan 100 en el 'SIMILARITY'
df_pre = df_pre[df_pre['SIMILARITY'] < 100]

In [8]:
df_new = pd.read_csv('Coincidencias.csv')
df_new

,NOMBRE_INVENTOR,NOMBRE DEL INVESTIGADOR,Ratio
0,"LOPEZ,NYDIA","LOPEZ LIRA,NIDIA",73.0
1,"STEPHENS,CHRISTOPER R","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0
2,"STEPHENS,CHRISTOPER","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0
3,"GARCIA,MARCO","GARCIA JUAREZ,MARCOS",77.0
4,"MORENO,MARCOS","MORENO IBARRA,MARCO",77.0
...,...,...,...
934,"AGUILAR ARMENDARIZ,RAMON ALBERTO","AGUILAR ARMENDARIZ,RAMON ALBERTO",100.0
935,"SACRISTAN ROCK,EMILIO","SACRISTAN ROCK,EMILIO",100.0
936,"ALONSO VILATELA,MARIA ELISA","ALONSO VILATELA,MARIA ELISA",100.0
937,"VILLARREAL TREVINO,JUAN ANTONIO","VILLARREAL TREVINO,JUAN ANTONIO",100.0


In [ ]:
df_PATENTE = pd.read_csv('listaNombresPatentes.csv')
df_PATENTE

In [ ]:
df_SNII = pd.read_csv('Nombres_Limpios_Final.csv')
df_SNII

In [ ]:
df_Prenew = pd.read_csv('Pre Coincidencias.csv')
df_Prenew

In [ ]:
# df_baseF = pd.read_csv('datasetF.csv')
# df_baseF

In [ ]:
# df_baseP = pd.read_csv('InventoresF.csv')
# df_baseP